#### **1. INTRODUCTION**

This notebook presents a solution to the [Playground Series - Season 5, Episode 9](https://www.kaggle.com/competitions/playground-series-s5e9) Kaggle competition, held in September 2025. The goal is to predict a song's beats-per-minute, with submissions evaluated using the Root Mean Squared Error (RMSE) between predicted and observed targets.

The workflow begins with importing the necessary libraries, followed by loading the training and testing datasets. A basic exploratory data analysis (EDA) is then performed, including examining shapes, structure, summary statistics, and other key information for both DataFrames.

Next, feature engineering is carried out. Pairwise and triplet combinations of columns are created to generate additional features, and the quartile and decile for each column's values are computed.

In the modeling stage, XGBoost and LightGBM models are defined with appropriate hyperparameters and trained using 5-fold cross-validation. In each fold, models are trained on the training set, and predictions are made for both the validation fold and the test data (the latter averaged across folds). Out-of-fold predictions are then compared to the true target values to calculate cross-validation RMSE.

Finally, predictions from both models are blended by averaging, and a new cross-validation RMSE is computed. A CSV file containing the averaged test set predictions is created for submission to the competition.

#### **2. IMPORT LIBRARIES**

First, we import all the libraries required for this notebook. NumPy is used for numerical operations and Pandas for data manipulation. Fore and Style from colorama are used to display exploratory data analysis. Combinations from itertools are imported to generate feature interactions during feature engineering. XGBRegressor and LGBMRegressor from xgboost and lightgbm, respectively, are used for model training. Finally, KFold and mean_squared_error from scikit-learn are used for cross-validation and regression evaluation.

In [1]:
# ===== IMPORT LIBRARIES =====
import numpy as np
import pandas as pd
from colorama import Fore, Style
from itertools import combinations
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

#### **3. LOAD DATA**

In this step, the training and testing datasets are loaded from files as pandas DataFrames. The id column is set as the index of both DataFrames to ensure unique identification and alignment.

In [2]:
# ===== LOAD DATA =====
X = pd.read_csv('/kaggle/input/playground-series-s5e9/train.csv').set_index('id')
X_test = pd.read_csv('/kaggle/input/playground-series-s5e9/test.csv').set_index('id')

#### **4. EXPLORE DATA**

Next, we perform basic exploratory data analysis (EDA), examining the shapes, heads, information, descriptions, and the number of unique and missing values for both the training and testing datasets.


In [3]:
# ===== EXPLORE DATA =====
def display_eda(dfs, names, target):
    print(f"{Fore.BLUE}{'-'*50}\n{names[0]} | SHAPE = {dfs[0].shape}\n{names[1]} | SHAPE = {dfs[1].shape}{Style.RESET_ALL}")
    
    for name, df in zip(names, dfs):
        print(f"{Fore.CYAN}{'-'*50}\n{name} head:{Style.RESET_ALL}")
        display(df.head().style.format(precision=3))
    
    print(f"{Fore.MAGENTA}{'-'*50}\nInformation and description{Style.RESET_ALL}")
    for i, (name, df) in enumerate(zip(names, dfs)):
        print(f"{Fore.BLUE}{'-'*50}\n{name} description:{Style.RESET_ALL}")
        display(df.drop(columns=[target], errors='ignore').describe().round(2))
        
        print(f"{Fore.BLUE}{'-'*50}\n{name} information:{Style.RESET_ALL}")
        display(df.info())
    
    print(f"{Fore.BLUE}{'-'*50}\nUnique and null values:{Style.RESET_ALL}")
    info_df = pd.concat([
        dfs[0].drop(columns=[target], errors='ignore').nunique(),
        dfs[1].drop(columns=[target], errors='ignore').nunique(),
        dfs[0].drop(columns=[target], errors='ignore').isna().sum(),
        dfs[1].drop(columns=[target], errors='ignore').isna().sum()
    ], axis=1)
    
    info_df.columns = ['Training_Nunq', 'Testing_Nunq', 'Training_Nulls', 'Testing_Nulls']
    display(info_df.T.style.format(formatter='{:.0f}'))

display_eda(dfs=[X, X_test], names=['Training data', 'Testing data'], target='BeatsPerMinute')

--------------------------------------------------
Training data | SHAPE = (524164, 10)
Testing data | SHAPE = (174722, 9)
--------------------------------------------------
Training data head:


,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
id,,,,,,,,,,
0,0.604,-7.637,0.024,0.000,0.000,0.051,0.410,290715.645,0.826,147.530
1,0.639,-16.268,0.072,0.445,0.349,0.171,0.651,164519.517,0.145,136.160
2,0.515,-15.954,0.111,0.174,0.454,0.030,0.424,174495.567,0.625,55.320
3,0.734,-1.357,0.053,0.002,0.160,0.086,0.279,225567.465,0.487,147.912
4,0.533,-13.056,0.024,0.069,0.000,0.331,0.478,213960.679,0.947,89.585


--------------------------------------------------
Testing data head:


,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
id,,,,,,,,,
524164,0.410,-16.795,0.024,0.233,0.013,0.272,0.664,302901.550,0.425
524165,0.463,-1.357,0.142,0.058,0.258,0.098,0.830,221995.664,0.846
524166,0.687,-3.369,0.168,0.288,0.211,0.326,0.305,357724.013,0.134
524167,0.886,-5.598,0.118,0.000,0.377,0.134,0.488,271790.399,0.316
524168,0.637,-7.068,0.126,0.539,0.069,0.024,0.591,277728.538,0.481


--------------------------------------------------
Information and description
--------------------------------------------------
Training data description:


...

,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
count,174722.00,174722.00,174722.00,174722.00,174722.00,174722.00,174722.00,174722.00,174722.00
mean,0.63,-8.38,0.07,0.26,0.12,0.18,0.56,241753.74,0.50
std,0.16,4.62,0.05,0.22,0.13,0.12,0.23,59103.90,0.29
min,0.14,-27.44,0.02,0.00,0.00,0.02,0.03,63973.00,0.00
25%,0.51,-11.55,0.02,0.07,0.00,0.08,0.40,207518.15,0.25
50%,0.63,-8.25,0.07,0.24,0.07,0.17,0.57,243584.59,0.51
75%,0.74,-4.90,0.11,0.40,0.20,0.27,0.72,281737.45,0.75
max,0.98,-1.36,0.26,1.00,0.68,0.60,0.98,449288.81,1.00


--------------------------------------------------
Testing data information:
<class 'pandas.core.frame.DataFrame'>
Index: 174722 entries, 524164 to 698885
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   RhythmScore                174722 non-null  float64
 1   AudioLoudness              174722 non-null  float64
 2   VocalContent               174722 non-null  float64
 3   AcousticQuality            174722 non-null  float64
 4   InstrumentalScore          174722 non-null  float64
 5   LivePerformanceLikelihood  174722 non-null  float64
 6   MoodScore                  174722 non-null  float64
 7   TrackDurationMs            174722 non-null  float64
 8   Energy                     174722 non-null  float64
dtypes: float64(9)
memory usage: 13.3 MB


None

--------------------------------------------------
Unique and null values:


,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
Training_Nunq,322528,310411,229305,270478,218979,279591,306504,377442,11606
Testing_Nunq,116151,110402,84370,97364,79221,101149,109993,133624,10465
Training_Nulls,0,0,0,0,0,0,0,0,0
Testing_Nulls,0,0,0,0,0,0,0,0,0


#### **5. PREPARE FEATURES**

This section focuses on feature engineering before modeling. An additional function, add_features, is used for this purpose.

First, a dictionary called new_features is created to store the generated columns. We loop over all pairwise combinations of existing features, performing multiplication and division (adding 1e-6 to denominators to avoid division by zero). Another loop generates new columns by multiplying all triplet combinations of features.

Next, for each column, two features are created to indicate the quartile and decile in which the values fall. This is done using pd.cut with 4 and 10 bins, respectively, without labels to produce integers, and including the lowest value. This works because all columns are numerical and contain no missing values.

Finally, the dictionary with the created features is converted to a DataFrame with the same index as the training or testing dataset, to be concatenated as new columns.

Before applying this function to both training and testing datasets, the target variable is stored separately and removed from the training dataset to ensure it is excluded from feature calculations.


In [4]:
# ===== PREPARE FEATURES =====
def add_features(df):
    new_features = {}
    for col1, col2 in list(combinations(df.columns, 2)):
        new_features[f"{col1}_m_{col2}"] = df[col1] * df[col2]
        new_features[f"{col1}_d_{col2}"] = df[col1] / (df[col2] + 1e-6)

    for col1, col2, col3 in list(combinations(df.columns, 3)):
        new_features[f"{col1}_m_{col2}_m_{col3}"] = df[col1] * df[col2] * df[col3]

    for col in df.columns:
        new_features[f"{col}_quartile"] = pd.cut(df[col], bins=4, labels=False, include_lowest=True)
        new_features[f"{col}_decile"] = pd.cut(df[col], bins=10, labels=False, include_lowest=True)

    df = pd.concat([df, pd.DataFrame(new_features, index=df.index)], axis=1)
    return df

y = X['BeatsPerMinute']
X = X.drop(['BeatsPerMinute'], axis=1)

X = add_features(X)
X_test = add_features(X_test)

#### **6. XGBOOST MODEL**

In this step, the XGBoost model is defined. The objective is set to regression, using squared error as the loss function, and the evaluation metric during training and validation is root mean squared error (RMSE). A total of 1000 trees are used to allow sufficient learning without overfitting, with each tree limited to a maximum depth of 6 to control complexity. A learning rate of 0.002 ensures slow and stable learning.

For each tree, two-thirds of the features are used, and for each node, two-thirds of the features are sampled via the colsample_bytree and colsample_bynode parameters. L1 and L2 regularization are applied by setting reg_alpha to 2.50 and reg_lambda to 0.85 to penalize large leaf outputs. Finally, a random state is set for reproducibility.

In [5]:
# ===== XGBOOST MODEL =====
xgb = XGBRegressor(
    objective = 'reg:squarederror',
    eval_metric = 'rmse',
    n_estimators = 1000,
    max_depth = 6,
    learning_rate = 0.002,
    colsample_bytree = 0.67,
    colsample_bynode = 0.67,
    reg_alpha = 2.50,
    reg_lambda = 0.85,
    random_state = 42
)

#### **7. LIGHTGBM MODEL**

In this step, we define the hyperparameters for the LightGBM model. A total of 1000 trees are used to allow sufficient learning without overfitting. Each tree’s depth is limited to 14, and the number of leaves is limited to 85, which is relatively high and allows the model to capture complex interactions. A low learning rate of 0.0015 ensures slow and stable learning.

For each tree, 90% of the features and 90% of the rows are used, controlled by the feature_fraction and subsample parameters. This introduces randomness and improves generalization. Large leaf outputs are slightly penalized by setting reg_alpha and reg_lambda to 0.0001. Finally, a random state is set for reproducibility, and verbosity is set to -1 to silence output during training.

In [6]:
# ===== LIGHTGBM MODEL =====
lgbm = LGBMRegressor(
    n_estimators = 1000,
    max_depth = 14,
    num_leaves = 85,
    learning_rate = 0.0015,
    feature_fraction = 0.90,
    subsample = 0.90,
    reg_alpha = 0.0001,
    reg_lambda = 0.0001,
    random_state = 42,
    verbosity = -1
)

#### **8. 5-FOLD CROSS-VALIDATION**

We use 5-fold cross-validation to train the models. First, two dictionaries are created to store predictions: oof_preds will hold out-of-fold predictions on the training data, and test_preds will store predictions on the testing data, averaged across folds. Each dictionary has a key for each model, with values as NumPy arrays of appropriate lengths initialized to zero.

Next, an instance of KFold is created with 5 splits, shuffling enabled to promote generalization, and a fixed random state for reproducibility.

For each fold, the data is split into training and validation sets. The XGBoost and LightGBM models are fitted on the training set, with XGBoost’s verbose output silenced. We then loop over both models to generate out-of-fold predictions on the validation set and predictions on the test data, which are averaged across folds.

After all folds are completed, the out-of-fold predictions of both models are compared with the true targets, and the cross-validation RMSE is calculated.

In [15]:
# ===== 5-FOLD CROSS-VALIDATION =====
oof_preds = {'XGBoost': np.zeros(len(X)), 'LightGBM': np.zeros(len(X))}
test_preds = {'XGBoost': np.zeros(len(X_test)), 'LightGBM': np.zeros(len(X_test))}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for (train_idx, valid_idx) in kf.split(X, y):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    xgb.fit(X_train, y_train, verbose=False)
    lgbm.fit(X_train, y_train)

    for name, model in [('XGBoost', xgb), ('LightGBM', lgbm)]:
        oof_preds[name][valid_idx] = model.predict(X_valid)
        test_preds[name] += model.predict(X_test) / kf.n_splits

for model in ['XGBoost', 'LightGBM']:
    rmse = np.sqrt(mean_squared_error(y, oof_preds[model]))
    print(f'{Fore.RED}{model} predictions - RMSE score: {rmse:.6f}{Style.RESET_ALL}')

XGBoost predictions - RMSE score: 26.461237
LightGBM predictions - RMSE score: 26.460378


#### **9. BLENDING**

In this step, both the out-of-fold predictions and the test-set predictions are averaged across the two models. We then calculate the new cross-validation RMSE by comparing the averaged out-of-fold predictions with the true target values.

In [17]:
# ===== BLENDING =====
avg_oof_preds = (oof_preds['XGBoost'] + oof_preds['LightGBM']) / 2
avg_test_preds = (test_preds['XGBoost'] + test_preds['LightGBM']) / 2

rmse = np.sqrt(mean_squared_error(y, avg_oof_preds))
print(f'{Fore.RED}Averaged predictions - RMSE score: {rmse:.6f}{Style.RESET_ALL}')

Averaged predictions - RMSE score: 26.460300


#### **10. CREATE SUBMISSION FILE**

The final step is creating a CSV file for submission to the competition.

In [18]:
# ===== CREATE SUBMISSION FILE =====
output = pd.DataFrame({'id': X_test.index, 'y': avg_test_preds})
output.to_csv('submission.csv', index=False)